<a href="https://colab.research.google.com/github/tourihasi/Openstudio/blob/main/%E5%90%84%E3%82%AB%E3%83%86%E3%82%B4%E3%83%AA%E8%A6%81%E7%B4%A0%E3%81%AE%E6%8A%BD%E5%87%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install ifcopenshell eppy geomeppy shapely

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.7/869.7 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 105.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 64.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packag

In [ ]:
from google.colab import files

print("IFCファイルを選択してください")
uploaded_ifc = files.upload()

print("Energy+.idd を選択してください")
uploaded_idd = files.upload()


IFCファイルを選択してください


Saving input.ifc to input.ifc
Energy+.idd を選択してください


In [ ]:
# -*- coding: utf-8 -*-
"""
IFC 解析 → CSV 出力（壁/床/屋根/開口/窓/ドア/カーテンウォール + リレーション）
- Revit 由来IFCを想定
- 幾何統計: 三角メッシュから面積・重心・代表法線・BBox など
- リレーション: IfcRelVoidsElement（壁⇔Opening）, IfcRelFillsElement（Opening⇔Window/Door）
- 追加診断: 窓/ドア/カーテンウォール → 最近傍の壁平面（幾何ベース）候補マッチ

依存: pip install ifcopenshell shapely pandas numpy
"""

import os, sys, math, json
import numpy as np
import pandas as pd

# ============ ユーザ設定 ============
IFC_PATH    = "/content/input.ifc"            # 解析するIFC
OUTPUT_DIR  = "/content/ifc_csv"              # CSVの出力先フォルダ
PLANE_ROUND = 5                                # 平面キー丸め桁（幾何マッチ用）
AREA_EPS    = 1e-6
Z_UP        = np.array([0.0, 0.0, 1.0])

# ============ 依存ライブラリ ============
try:
    import ifcopenshell
    import ifcopenshell.geom as ifc_geom
except Exception as e:
    raise RuntimeError("ifcopenshell が必要です。pip install ifcopenshell") from e
try:
    from shapely.geometry import Polygon, MultiPolygon
    from shapely.ops import unary_union
except Exception as e:
    raise RuntimeError("shapely が必要です。pip install shapely") from e

# ============ ユーティリティ ============
def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

def normalize(v):
    n = np.linalg.norm(v)
    return v / n if n else v

def tri_normal(p0, p1, p2):
    return normalize(np.cross(p1 - p0, p2 - p0))

def plane_key(p, n, rnd=PLANE_ROUND):
    n = normalize(n)
    if n[2] < 0:  # Z優先で向きを正規化（符号反転で同一視）
        n = -n
    d = -np.dot(n, p)  # n·x + d = 0
    return (round(n[0], rnd), round(n[1], rnd), round(n[2], rnd), round(d, rnd))

def bbox_from_points(P):
    """P: (N,3) -> bbox_min(3), bbox_max(3)"""
    if P.size == 0:
        return [np.nan]*3, [np.nan]*3
    mn = np.min(P, axis=0)
    mx = np.max(P, axis=0)
    return mn.tolist(), mx.tolist()

def mesh_stats(verts, faces):
    """
    面積、重心（頂点平均ベースの簡易）、代表法線（面法線の面積重み平均）を返す。
    verts: (V,3), faces: (F,3) int
    """
    if verts.size == 0 or faces.size == 0:
        return 0.0, [np.nan]*3, [np.nan]*3
    area_sum = 0.0
    n_accum = np.zeros(3)
    c_accum = np.zeros(3)
    for a, b, c in faces:
        p0, p1, p2 = verts[a], verts[b], verts[c]
        # 三角形面積
        tri_area = 0.5 * np.linalg.norm(np.cross(p1 - p0, p2 - p0))
        area_sum += tri_area
        # 法線を面積重みで積算
        n_accum += tri_area * tri_normal(p0, p1, p2)
        # 簡易重心
        c_accum += (p0 + p1 + p2) / 3.0
    n_rep = normalize(n_accum) if np.linalg.norm(n_accum) > 0 else np.array([np.nan, np.nan, np.nan])
    c_rep = (c_accum / len(faces)).tolist()
    return float(area_sum), c_rep, n_rep.tolist()

def class_surface_from_normal(nz, tilt_roof=0.3, tilt_floor=-0.3):
    """
    代表法線z成分から簡易分類（水平近傍=屋根/床、垂直近傍=壁）
    nz: 代表法線z
    """
    try:
        if nz >= tilt_roof:   return "RoofLike"
        if nz <= tilt_floor:  return "FloorLike"
        return "WallLike"
    except Exception:
        return "Unknown"

def init_geom_settings():
    s = ifc_geom.settings()
    # 安定のための基本設定（Revit由来IFCで広く無難）
    def set_opt(key, val):
        try:
            s.set(getattr(s, key), val)
        except Exception:
            pass
    set_opt("USE_WORLD_COORDS", True)
    set_opt("INCLUDE_CURVES", False)
    set_opt("DISABLE_TRIANGULATION", False)
    return s

# ============ ジオメトリ抽出 ============
def shape_of(el, settings):
    try:
        shp = ifc_geom.create_shape(settings, el)
        verts = np.array(shp.geometry.verts, dtype=float).reshape((-1, 3))
        faces = np.array(shp.geometry.faces, dtype=int).reshape((-1, 3))
        return verts, faces
    except Exception:
        return np.zeros((0,3)), np.zeros((0,3), dtype=int)

def element_rows(ifc, settings, ifc_types, kind_label):
    """
    指定IFCタイプ群について、要素ごとに幾何統計・メタデータを収集し辞書のリストで返す。
    kind_label: 出力 CSV での論理カテゴリ名（'Wall','Slab','Opening','WindowLike' 等）
    """
    rows = []
    for t in ifc_types:
        for el in ifc.by_type(t):
            guid = getattr(el, "GlobalId", None)
            name = getattr(el, "Name", None)
            pset  = getattr(el, "PredefinedType", None) if hasattr(el, "PredefinedType") else None
            verts, faces = shape_of(el, settings)
            area, centroid, nrep = mesh_stats(verts, faces)
            bbmin, bbmax = bbox_from_points(verts)
            nz = nrep[2] if isinstance(nrep, list) and len(nrep) == 3 else np.nan
            surf_class = class_surface_from_normal(nz)
            # 主だった平面キー（最初の三角の平面）
            if faces.size:
                a, b, c = faces[0]
                pk = plane_key(verts[a], tri_normal(verts[a], verts[b], verts[c]), rnd=PLANE_ROUND)
            else:
                pk = (np.nan, np.nan, np.nan, np.nan)

            rows.append(dict(
                kind=kind_label,
                ifc_type=t,
                ifc_id=el.id(),
                guid=guid,
                name=name,
                predefined_type=str(pset) if pset else "",
                area_m2=area,
                centroid_x=centroid[0], centroid_y=centroid[1], centroid_z=centroid[2],
                nrep_x=nrep[0], nrep_y=nrep[1], nrep_z=nrep[2],
                nz_class=surf_class,
                bbox_min_x=bbmin[0], bbox_min_y=bbmin[1], bbox_min_z=bbmin[2],
                bbox_max_x=bbmax[0], bbox_max_y=bbmax[1], bbox_max_z=bbmax[2],
                plane_key_nx=pk[0], plane_key_ny=pk[1], plane_key_nz=pk[2], plane_key_d=pk[3],
            ))
    return rows

# ============ リレーション抽出（Voids/Fills） ============
def extract_relations(ifc):
    """
    - IfcRelVoidsElement: RelatingBuildingElement(壁/スラブ等) ←→ RelatedOpeningElement(Opening)
    - IfcRelFillsElement: RelatingOpeningElement(Opening) ←→ RelatedBuildingElement(Window/Door)
    """
    rel_voids_rows = []
    for rel in ifc.by_type("IfcRelVoidsElement"):
        be = getattr(rel, "RelatingBuildingElement", None)
        oe = getattr(rel, "RelatedOpeningElement", None)
        rel_voids_rows.append(dict(
            rel_id=rel.id(),
            rel_type="IfcRelVoidsElement",
            building_elem_id=getattr(be, "id", lambda: None)(),
            building_elem_type=getattr(be, "is_a", lambda: None)(),
            opening_elem_id=getattr(oe, "id", lambda: None)(),
            opening_elem_type=getattr(oe, "is_a", lambda: None)(),
        ))

    rel_fills_rows = []
    for rel in ifc.by_type("IfcRelFillsElement"):
        oe = getattr(rel, "RelatingOpeningElement", None)
        be = getattr(rel, "RelatedBuildingElement", None)  # Window/Door
        rel_fills_rows.append(dict(
            rel_id=rel.id(),
            rel_type="IfcRelFillsElement",
            opening_elem_id=getattr(oe, "id", lambda: None)(),
            opening_elem_type=getattr(oe, "is_a", lambda: None)(),
            filling_elem_id=getattr(be, "id", lambda: None)(),
            filling_elem_type=getattr(be, "is_a", lambda: None)(),
        ))
    return rel_voids_rows, rel_fills_rows

# ============ カーテンウォール等の“窓扱い”候補をまとめる ============
WINDOW_LIKE_TYPES = ["IfcWindow", "IfcCurtainWall", "IfcPlate", "IfcMember"]
DOOR_TYPES        = ["IfcDoor"]
WALL_TYPES        = ["IfcWallStandardCase", "IfcWall"]
SLAB_TYPES        = ["IfcSlab"]   # PredefinedType = FLOOR/ROOF など判定に利用
OPENING_TYPES     = ["IfcOpeningElement"]

def collect_all(ifc):
    settings = init_geom_settings()

    # 要素の幾何テーブル
    rows = []
    rows += element_rows(ifc, settings, WALL_TYPES,        "Wall")
    rows += element_rows(ifc, settings, SLAB_TYPES,        "Slab")
    rows += element_rows(ifc, settings, OPENING_TYPES,     "Opening")
    rows += element_rows(ifc, settings, WINDOW_LIKE_TYPES, "WindowLike")
    rows += element_rows(ifc, settings, DOOR_TYPES,        "DoorLike")

    df_elems = pd.DataFrame(rows).sort_values(["kind", "ifc_type", "area_m2"], ascending=[True, True, False])

    # リレーション（Voids/Fills）
    rel_voids_rows, rel_fills_rows = extract_relations(ifc)
    df_voids = pd.DataFrame(rel_voids_rows)
    df_fills = pd.DataFrame(rel_fills_rows)

    return df_elems, df_voids, df_fills

# ============ 追加診断：幾何マッチ（窓/ドア → 近い壁平面候補） ============
def build_wall_plane_index(df_elems):
    """平面キーごとに壁候補を束ねる（代表三角の plane_key ベース）"""
    walls = df_elems[df_elems["kind"]=="Wall"].copy()
    walls["pk"] = list(zip(walls["plane_key_nx"], walls["plane_key_ny"], walls["plane_key_nz"], walls["plane_key_d"]))
    idx = {}
    for _, r in walls.iterrows():
        idx.setdefault(r["pk"], []).append(r)
    return idx, walls

def nearest_wall_candidates(df_elems, topk=3):
    """
    WindowLike/DoorLike に対し、(1) plane_key一致、(2) bbox中心のユークリッド距離 が近い壁を候補提示。
    """
    idx, walls = build_wall_plane_index(df_elems)
    rows = []

    def row_center(r):
        cx = 0.5*(r["bbox_min_x"] + r["bbox_max_x"])
        cy = 0.5*(r["bbox_min_y"] + r["bbox_max_y"])
        cz = 0.5*(r["bbox_min_z"] + r["bbox_max_z"])
        return np.array([cx, cy, cz])

    fen = df_elems[df_elems["kind"].isin(["WindowLike","DoorLike"])].copy()
    fen["pk"] = list(zip(fen["plane_key_nx"], fen["plane_key_ny"], fen["plane_key_nz"], fen["plane_key_d"]))

    for _, fr in fen.iterrows():
        fcenter = row_center(fr)
        # (a) plane_key 一致候補
        cands = idx.get(fr["pk"], [])
        # (b) plane_key が無効な場合は全壁から距離順
        if not cands:
            # 全壁から距離上位
            walls["dist"] = ( ( (walls["bbox_min_x"]+walls["bbox_max_x"])/2 - fcenter[0])**2
                            + ( (walls["bbox_min_y"]+walls["bbox_max_y"])/2 - fcenter[1])**2
                            + ( (walls["bbox_min_z"]+walls["bbox_max_z"])/2 - fcenter[2])**2 )**0.5
            cands = walls.nsmallest(topk, "dist").to_dict("records")
        else:
            # plane_key一致の中から距離順
            cand_df = pd.DataFrame(cands)
            cand_df["dist"] = ( ( (cand_df["bbox_min_x"]+cand_df["bbox_max_x"])/2 - fcenter[0])**2
                              + ( (cand_df["bbox_min_y"]+cand_df["bbox_max_y"])/2 - fcenter[1])**2
                              + ( (cand_df["bbox_min_z"]+cand_df["bbox_max_z"])/2 - fcenter[2])**2 )**0.5
            cands = cand_df.nsmallest(topk, "dist").to_dict("records")

        for c in cands:
            rows.append(dict(
                fen_ifc_id=fr["ifc_id"],
                fen_guid=fr["guid"],
                fen_kind=fr["kind"],
                fen_ifc_type=fr["ifc_type"],
                fen_area_m2=fr["area_m2"],
                fen_pk=str(fr["pk"]),
                wall_ifc_id=c["ifc_id"],
                wall_guid=c["guid"],
                wall_area_m2=c["area_m2"],
                wall_pk=str(c["pk"]),
                center_dist=np.linalg.norm(row_center(c)-fcenter) if isinstance(c, dict) else c["dist"]
            ))
    return pd.DataFrame(rows)

# ============ メイン：CSV出力 ============
def main():
    assert os.path.isfile(IFC_PATH), f"IFC not found: {IFC_PATH}"
    ensure_dir(OUTPUT_DIR)

    ifc = ifcopenshell.open(IFC_PATH)

    df_elems, df_voids, df_fills = collect_all(ifc)

    # CSV 書き出し
    df_elems.to_csv(os.path.join(OUTPUT_DIR, "elements_geometry.csv"), index=False)
    df_voids.to_csv(os.path.join(OUTPUT_DIR, "relations_voids.csv"), index=False)
    df_fills.to_csv(os.path.join(OUTPUT_DIR, "relations_fills.csv"), index=False)

    # 幾何マッチ候補（窓/ドア → 壁）
    df_match = nearest_wall_candidates(df_elems, topk=5)
    df_match.to_csv(os.path.join(OUTPUT_DIR, "fenestration_wall_candidates.csv"), index=False)

    # 集計の味見（種類別・クラス別）
    summary_kind = (df_elems
                    .groupby(["kind","ifc_type","nz_class"], dropna=False)
                    .agg(count=("ifc_id","count"), sum_area_m2=("area_m2","sum"))
                    .reset_index())
    summary_kind.to_csv(os.path.join(OUTPUT_DIR, "summary_by_kind.csv"), index=False)

    print("=== Exported CSVs ===")
    for fn in ["elements_geometry.csv", "relations_voids.csv", "relations_fills.csv",
               "fenestration_wall_candidates.csv", "summary_by_kind.csv"]:
        print(os.path.join(OUTPUT_DIR, fn))

if __name__ == "__main__":
    main()


=== Exported CSVs ===
/content/ifc_csv/elements_geometry.csv
/content/ifc_csv/relations_voids.csv
/content/ifc_csv/relations_fills.csv
/content/ifc_csv/fenestration_wall_candidates.csv
/content/ifc_csv/summary_by_kind.csv
